# Library import

In [57]:
import pandas as pd
import numpy as np

import bs4 as bs
import pickle
import requests
import time
import lxml
from datetime import datetime, timedelta

import yfinance as yf
import stockstats

import unicodedata
import json
from textblob import TextBlob

# Global Variables

In [58]:
stock_date_start = "2009-11-29"
stock_date_end = "2019-12-31"
target = "MSFT"

# 1. Data Preprocessing

In [59]:
stock_raw = yf.download(target, start=stock_date_start, end=stock_date_end)

[*********************100%***********************]  1 of 1 completed


In [60]:
stock_raw.transpose()

Date,2009-11-30,2009-12-01,2009-12-02,2009-12-03,2009-12-04,2009-12-07,2009-12-08,2009-12-09,2009-12-10,2009-12-11,...,2019-12-16,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30
Open,2.915000e+01,2.952000e+01,2.990000e+01,2.984000e+01,3.005000e+01,2.978000e+01,2.952000e+01,2.947000e+01,2.971000e+01,2.997000e+01,...,1.551100e+02,1.554500e+02,1.543000e+02,1.540000e+02,1.573500e+02,1.581200e+02,1.574800e+02,1.575600e+02,1.594500e+02,1.589900e+02
High,2.945000e+01,3.005000e+01,2.999000e+01,3.020000e+01,3.037000e+01,3.008000e+01,2.974000e+01,2.981000e+01,2.996000e+01,3.000000e+01,...,1.559000e+02,1.557100e+02,1.554800e+02,1.557700e+02,1.584900e+02,1.581200e+02,1.577100e+02,1.587300e+02,1.595500e+02,1.590200e+02
Low,2.900000e+01,2.941000e+01,2.965000e+01,2.976000e+01,2.983000e+01,2.968000e+01,2.938000e+01,2.925000e+01,2.966000e+01,2.979000e+01,...,1.548200e+02,1.544500e+02,1.541800e+02,1.537500e+02,1.562900e+02,1.572700e+02,1.571200e+02,1.574000e+02,1.582200e+02,1.567300e+02
Close,2.941000e+01,3.001000e+01,2.978000e+01,2.983000e+01,2.998000e+01,2.979000e+01,2.957000e+01,2.971000e+01,2.987000e+01,2.985000e+01,...,1.555300e+02,1.546900e+02,1.543700e+02,1.557100e+02,1.574100e+02,1.574100e+02,1.573800e+02,1.586700e+02,1.589600e+02,1.575900e+02
Adj Close,2.308554e+01,2.355651e+01,2.337597e+01,2.341522e+01,2.353296e+01,2.338382e+01,2.321113e+01,2.332102e+01,2.344662e+01,2.343092e+01,...,1.551063e+02,1.542686e+02,1.539495e+02,1.552859e+02,1.569812e+02,1.569812e+02,1.569513e+02,1.582378e+02,1.585270e+02,1.571607e+02
Volume,4.417200e+07,4.990420e+07,3.630860e+07,4.309520e+07,5.881070e+07,3.808270e+07,3.740220e+07,4.471330e+07,4.594020e+07,4.374420e+07,...,2.414420e+07,2.542560e+07,2.412920e+07,2.495890e+07,5.347750e+07,1.771820e+07,8.989200e+06,1.452060e+07,1.841280e+07,1.634840e+07


## Financial Indicators Features

In [61]:
# origin data features: ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
stock_raw_data = stock_raw
# Add high_low_diff, open_close_diff, high_low_diff_ratio, open_close_diff_ratio
stock_raw_data['high_low_diff'] = (stock_raw_data['High'] - stock_raw_data['Low'])
stock_raw_data['open_close_diff'] = (stock_raw_data['Open'] - stock_raw_data['Close'])
stock_raw_data['high_low_diff_ratio'] = (stock_raw_data['High'] - stock_raw_data['Low']) / stock_raw_data['Close']
stock_raw_data['open_close_diff_ratio'] = (stock_raw_data['Open'] - stock_raw_data['Close']) / stock_raw_data['Close']

# Add financial indicators
stock_stats_data = stockstats.StockDataFrame.retype(stock_raw_data)
stock_stats_data[['close_1_d', 'close_7_d', 'close_30_d','change', 'open_delta','close_delta','volume_delta', 
       'close_-2_r','close_-6_r', 'boll', 'boll_ub', 'boll_lb', 'boll_-1_d', 'boll_ub_-1_d', 'boll_lb_-1_d' , 
       'kdjk','kdjd','kdjj', 'macd','macds','macdh', 'rsi_6', 'rsi_12', 'wr_6', 'wr_12', 'cci', 'atr', 'dma', 'vr']]

# Add financial indicators
stock_data = pd.DataFrame(stock_stats_data)
stock_data = stock_data.dropna()
stock_data['boll_k_diff'] = stock_data['boll'] - stock_data['close']

# Generate absolute label (namely 0,1) and non-absolute label (real price)
label_abs_1d = stock_data['close_1_d'].apply(lambda x: 1 if x > 0 else 0)
label_abs_7d = stock_data['close_7_d'].apply(lambda x: 1 if x > 0 else 0)
label_abs_30d = stock_data['close_30_d'].apply(lambda x: 1 if x > 0 else 0)

label_value_1d = stock_data['close_1_s']
label_value_7d = stock_data['close_7_s']
label_value_30d = stock_data['close_30_s']

# training datasets of with or without absolute data
stock_without_absolute = stock_data[['change', 'open_delta','close_delta','volume_delta', 'high_low_diff_ratio', 
                                 'open_close_diff_ratio','close_-2_r','close_-6_r','kdjk','kdjd','kdjj', 'macd',
                                 'macds', 'macdh', 'rsi_6', 'rsi_12', 'wr_6', 'wr_12', 'cci', 'atr', 'dma', 'vr', 
                                 'boll_-1_d','boll_ub_-1_d', 'boll_lb_-1_d', 'boll_k_diff',
                                 'high_low_diff', 'open_close_diff']]

stock_with_absolute = stock_data[['change', 'open_delta','close_delta','volume_delta', 'high_low_diff_ratio', 
                                'open_close_diff_ratio','close_-2_r','close_-6_r','kdjk','kdjd','kdjj', 'macd',
                                'macds', 'macdh', 'rsi_6', 'rsi_12', 'wr_6', 'wr_12', 'cci', 'atr', 'dma', 'vr', 
                                'boll_-1_d','boll_ub_-1_d', 'boll_lb_-1_d', 'boll_k_diff',
                                'high_low_diff', 'open_close_diff', 
                                'open', 'high', 'low', 'close', 'adj close', 'volume', ]]
print("Process data from: ", stock_with_absolute.index[0], " to ", stock_with_absolute.index[-1])
final_index = stock_with_absolute

/Users/chinkashiwakin/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
NOTE: Behavior of MACDH calculation has changed as of July 2017 - it is now 1/2 of previous calculated values


Process data from:  2009-12-08 00:00:00  to  2019-11-14 00:00:00


In [62]:
stock_with_absolute.describe()

,change,open_delta,close_delta,volume_delta,high_low_diff_ratio,open_close_diff_ratio,close_-2_r,close_-6_r,kdjk,kdjd,...,boll_lb_-1_d,boll_k_diff,high_low_diff,open_close_diff,open,high,low,close,adj close,volume
count,2502.000000,2502.000000,2502.000000,2.502000e+03,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,...,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2.502000e+03
mean,0.074439,0.046859,0.047270,-7.335292e+03,0.017341,-0.000235,0.147704,0.436701,57.119822,57.106070,...,0.042618,-0.437239,0.942222,-0.010260,54.937342,55.391143,54.448921,54.947602,50.905797,4.016584e+07
std,1.438169,0.879032,0.911218,2.020780e+07,0.009083,0.011360,1.994770,3.286971,22.227605,19.199233,...,0.273800,1.724600,0.811377,0.738079,32.262246,32.498733,31.957023,32.245552,33.439694,2.298603e+07
min,-11.399546,-6.469994,-6.099998,-2.294923e+08,0.004120,-0.053898,-12.380344,-14.475906,5.849243,11.387073,...,-2.261275,-8.076999,0.150000,-5.419998,23.090000,23.320000,22.730000,23.010000,18.228165,7.425600e+06
25%,-0.674071,-0.290001,-0.280005,-6.549600e+06,0.011445,-0.006787,-0.931561,-1.365578,40.394887,43.085020,...,-0.051839,-1.213751,0.459999,-0.310001,29.530001,29.740000,29.180000,29.512500,24.267555,2.459455e+07
50%,0.053514,0.030001,0.020000,-4.194500e+05,0.015230,-0.000389,0.192823,0.529260,59.434207,58.800491,...,0.014187,-0.337751,0.669998,-0.019999,43.875000,44.195000,43.500000,43.920000,39.584156,3.454705e+07
75%,0.819278,0.370001,0.360001,6.349025e+06,0.020423,0.005703,1.261678,2.351975,76.371931,72.923998,...,0.125899,0.454500,1.110001,0.250000,68.907503,69.425001,68.479998,69.030001,65.731054,4.985110e+07
max,10.452235,5.170006,6.599998,2.391179e+08,0.085416,0.061138,12.950242,15.401249,95.333442,92.557744,...,2.654123,12.149503,7.070000,6.090004,147.020004,148.410004,147.000000,148.059998,147.155960,3.193179e+08


## News Sentiment Features

In [63]:
# First run financial_news_data.ipynb to scrape financial news

# open file
with open('./data/'+target+'/news_09-19.json') as json_file:
    news_data = json.load(json_file)

time_index = list(stock_with_absolute.index)
score = {}
des_score = {}
for time in time_index:
    yesterday = (time - timedelta(days=1)).strftime("%m/%d/%Y")
    if(yesterday in news_data):
        num_news = len(news_data[yesterday])
        sentiment = 0
        des_sentiment = 0
        for news in news_data[yesterday]:
            news_title = news['news_title'].replace('...', '')
            news_des = news['news_text'].encode("ascii", "ignore").decode("ascii").replace('...', '')
            blob = TextBlob(news_title)
            des_blob = TextBlob(news_des)
            sentiment += blob.sentiment.polarity
            des_sentiment += des_blob.sentiment.polarity
        score[time] = sentiment / num_news
        des_score[time] = des_sentiment / num_news
    else:
        score[time] = 0.0
        des_score[time] = 0.0
des_score_series = pd.Series(des_score)
des_score_series.name = "news_des_score"
title_score_series = pd.Series(score)
title_score_series.name = "news_title_score"
stock_without_absolute = stock_without_absolute.join([title_score_series, des_score_series])
stock_with_absolute = stock_with_absolute.join([title_score_series, des_score_series])

In [64]:
stock_with_absolute.describe()

,change,open_delta,close_delta,volume_delta,high_low_diff_ratio,open_close_diff_ratio,close_-2_r,close_-6_r,kdjk,kdjd,...,high_low_diff,open_close_diff,open,high,low,close,adj close,volume,news_title_score,news_des_score
count,2502.000000,2502.000000,2502.000000,2.502000e+03,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,...,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2.502000e+03,2502.000000,2502.000000
mean,0.074439,0.046859,0.047270,-7.335292e+03,0.017341,-0.000235,0.147704,0.436701,57.119822,57.106070,...,0.942222,-0.010260,54.937342,55.391143,54.448921,54.947602,50.905797,4.016584e+07,0.075608,0.087938
std,1.438169,0.879032,0.911218,2.020780e+07,0.009083,0.011360,1.994770,3.286971,22.227605,19.199233,...,0.811377,0.738079,32.262246,32.498733,31.957023,32.245552,33.439694,2.298603e+07,0.146826,0.127471
min,-11.399546,-6.469994,-6.099998,-2.294923e+08,0.004120,-0.053898,-12.380344,-14.475906,5.849243,11.387073,...,0.150000,-5.419998,23.090000,23.320000,22.730000,23.010000,18.228165,7.425600e+06,-0.714286,-0.750000
25%,-0.674071,-0.290001,-0.280005,-6.549600e+06,0.011445,-0.006787,-0.931561,-1.365578,40.394887,43.085020,...,0.459999,-0.310001,29.530001,29.740000,29.180000,29.512500,24.267555,2.459455e+07,0.000000,0.000000
50%,0.053514,0.030001,0.020000,-4.194500e+05,0.015230,-0.000389,0.192823,0.529260,59.434207,58.800491,...,0.669998,-0.019999,43.875000,44.195000,43.500000,43.920000,39.584156,3.454705e+07,0.034091,0.067210
75%,0.819278,0.370001,0.360001,6.349025e+06,0.020423,0.005703,1.261678,2.351975,76.371931,72.923998,...,1.110001,0.250000,68.907503,69.425001,68.479998,69.030001,65.731054,4.985110e+07,0.140000,0.157197
max,10.452235,5.170006,6.599998,2.391179e+08,0.085416,0.061138,12.950242,15.401249,95.333442,92.557744,...,7.070000,6.090004,147.020004,148.410004,147.000000,148.059998,147.155960,3.193179e+08,1.000000,1.000000


## External Features

### a) S&P 500

In [65]:
SP500_raw = yf.download("^GSPC", start=stock_date_start, end=stock_date_end)
SP500_raw_data = SP500_raw
# Add high_low_diff, open_close_diff, high_low_diff_ratio, open_close_diff_ratio
SP500_raw_data['high_low_diff'] = (SP500_raw['High'] - SP500_raw['Low'])
SP500_raw_data['open_close_diff'] = (SP500_raw['Open'] - SP500_raw['Close'])
SP500_raw_data['high_low_diff_ratio'] = (SP500_raw['High'] - SP500_raw['Low']) / SP500_raw['Close']
SP500_raw_data['open_close_diff_ratio'] = (SP500_raw['Open'] - SP500_raw['Close']) / SP500_raw['Close']

# Add financial indicators
SP500_stats_data = stockstats.StockDataFrame.retype(SP500_raw_data)
SP500_stats_data[['change','close_delta','volume_delta', 'close_-2_r','close_-6_r']]

SP500_stock = pd.DataFrame(SP500_stats_data).add_prefix('sp500_')

[*********************100%***********************]  1 of 1 completed


In [66]:
stock_without_absolute = stock_without_absolute.join(SP500_stock[['sp500_high_low_diff',
       'sp500_open_close_diff', 'sp500_high_low_diff_ratio',
       'sp500_open_close_diff_ratio', 'sp500_change', 'sp500_close_delta',
       'sp500_volume_delta', 'sp500_close_-2_r', 'sp500_close_-6_r']].loc[stock_with_absolute.index])
stock_with_absolute = stock_with_absolute.join(SP500_stock.loc[stock_with_absolute.index])

In [67]:
stock_without_absolute.describe()

,change,open_delta,close_delta,volume_delta,high_low_diff_ratio,open_close_diff_ratio,close_-2_r,close_-6_r,kdjk,kdjd,...,news_des_score,sp500_high_low_diff,sp500_open_close_diff,sp500_high_low_diff_ratio,sp500_open_close_diff_ratio,sp500_change,sp500_close_delta,sp500_volume_delta,sp500_close_-2_r,sp500_close_-6_r
count,2502.000000,2502.000000,2502.000000,2.502000e+03,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,...,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2.502000e+03,2502.000000,2502.000000
mean,0.074439,0.046859,0.047270,-7.335292e+03,0.017341,-0.000235,0.147704,0.436701,57.119822,57.106070,...,0.087938,19.113069,-0.443317,0.010407,-0.000232,0.045620,0.796715,-3.306515e+05,0.090709,0.269849
std,1.438169,0.879032,0.911218,2.020780e+07,0.009083,0.011360,1.994770,3.286971,22.227605,19.199233,...,0.127471,13.318713,15.476230,0.007214,0.008641,0.933130,17.339129,6.895554e+08,1.286836,2.116332
min,-11.399546,-6.469994,-6.099998,-2.294923e+08,0.004120,-0.053898,-12.380344,-14.475906,5.849243,11.387073,...,-0.750000,3.680054,-104.579834,0.001456,-0.044604,-6.663446,-113.189941,-4.995080e+09,-7.000929,-13.373268
25%,-0.674071,-0.290001,-0.280005,-6.549600e+06,0.011445,-0.006787,-0.931561,-1.365578,40.394887,43.085020,...,0.000000,10.739990,-8.179932,0.005631,-0.004394,-0.328627,-6.202454,-2.977825e+08,-0.496023,-0.700018
50%,0.053514,0.030001,0.020000,-4.194500e+05,0.015230,-0.000389,0.192823,0.529260,59.434207,58.800491,...,0.067210,15.419922,-1.010010,0.008488,-0.000533,0.060568,1.054993,-1.056500e+07,0.156177,0.459180
75%,0.819278,0.370001,0.360001,6.349025e+06,0.020423,0.005703,1.261678,2.351975,76.371931,72.923998,...,0.157197,23.122498,5.917480,0.012834,0.003075,0.506680,9.250000,3.114475e+08,0.773983,1.494375
max,10.452235,5.170006,6.599998,2.391179e+08,0.085416,0.061138,12.950242,15.401249,95.333442,92.557744,...,1.000000,125.219971,104.010010,0.090227,0.070588,4.959374,116.599854,4.299510e+09,6.428004,8.761593


### b) Gold

In [68]:
gold_raw = yf.download("GLD", start=stock_date_start, end=stock_date_end)
gold_raw_data = gold_raw
# Add high_low_diff, open_close_diff, high_low_diff_ratio, open_close_diff_ratio
gold_raw_data['high_low_diff'] = (gold_raw['High'] - gold_raw['Low'])
gold_raw_data['open_close_diff'] = (gold_raw['Open'] - gold_raw['Close'])
gold_raw_data['high_low_diff_ratio'] = (gold_raw['High'] - gold_raw['Low']) / gold_raw['Close']
gold_raw_data['open_close_diff_ratio'] = (gold_raw['Open'] - gold_raw['Close']) / gold_raw['Close']

# Add financial indicators
gold_stats_data = stockstats.StockDataFrame.retype(gold_raw_data)
gold_stats_data[['change','close_delta','volume_delta', 'close_-2_r','close_-6_r']]

gold_stock = pd.DataFrame(gold_stats_data).add_prefix('gold_')

[*********************100%***********************]  1 of 1 completed


In [69]:
stock_without_absolute = stock_without_absolute.join(gold_stock[['gold_high_low_diff', 'gold_open_close_diff',
       'gold_high_low_diff_ratio', 'gold_open_close_diff_ratio', 'gold_change',
       'gold_close_delta', 'gold_volume_delta', 'gold_close_-2_r',
       'gold_close_-6_r']].loc[stock_with_absolute.index])
stock_with_absolute = stock_with_absolute.join(gold_stock.loc[stock_with_absolute.index])

In [70]:
stock_without_absolute.describe()

,change,open_delta,close_delta,volume_delta,high_low_diff_ratio,open_close_diff_ratio,close_-2_r,close_-6_r,kdjk,kdjd,...,sp500_close_-6_r,gold_high_low_diff,gold_open_close_diff,gold_high_low_diff_ratio,gold_open_close_diff_ratio,gold_change,gold_close_delta,gold_volume_delta,gold_close_-2_r,gold_close_-6_r
count,2502.000000,2502.000000,2502.000000,2.502000e+03,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,...,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2.502000e+03,2502.000000,2502.000000
mean,0.074439,0.046859,0.047270,-7.335292e+03,0.017341,-0.000235,0.147704,0.436701,57.119822,57.106070,...,0.269849,1.203102,0.005460,0.009169,0.000066,0.012975,0.010172,-1.491535e+04,0.025194,0.068703
std,1.438169,0.879032,0.911218,2.020780e+07,0.009083,0.011360,1.994770,3.286971,22.227605,19.199233,...,2.116332,0.840345,0.889768,0.005766,0.006542,0.984758,1.331456,5.665652e+06,1.364206,2.355535
min,-11.399546,-6.469994,-6.099998,-2.294923e+08,0.004120,-0.053898,-12.380344,-14.475906,5.849243,11.387073,...,-13.373268,0.209999,-6.689995,0.001707,-0.050678,-8.780826,-12.639999,-4.815340e+07,-13.068524,-14.069760
25%,-0.674071,-0.290001,-0.280005,-6.549600e+06,0.011445,-0.006787,-0.931561,-1.365578,40.394887,43.085020,...,-0.700018,0.670006,-0.377499,0.005415,-0.002941,-0.490806,-0.620001,-2.728850e+06,-0.722545,-1.320687
50%,0.053514,0.030001,0.020000,-4.194500e+05,0.015230,-0.000389,0.192823,0.529260,59.434207,58.800491,...,0.459180,0.980003,0.004997,0.007678,0.000028,0.036447,0.049992,-2.600500e+05,0.057978,0.127849
75%,0.819278,0.370001,0.360001,6.349025e+06,0.020423,0.005703,1.261678,2.351975,76.371931,72.923998,...,1.494375,1.430000,0.379997,0.010896,0.003010,0.518419,0.660004,2.519625e+06,0.827276,1.611773
max,10.452235,5.170006,6.599998,2.391179e+08,0.085416,0.061138,12.950242,15.401249,95.333442,92.557744,...,8.761593,9.589996,8.900009,0.058372,0.054173,4.903838,5.970001,4.753860e+07,5.469986,8.979403


### c) Bonds

In [71]:
# 5 year bonds
y5bond_raw = yf.download("^FVX", start=stock_date_start, end=stock_date_end)
y5bond_raw_data = y5bond_raw
# Add high_low_diff, open_close_diff, high_low_diff_ratio, open_close_diff_ratio
y5bond_raw_data['high_low_diff'] = (y5bond_raw['High'] - y5bond_raw['Low'])
y5bond_raw_data['open_close_diff'] = (y5bond_raw['Open'] - y5bond_raw['Close'])
y5bond_raw_data['high_low_diff_ratio'] = (y5bond_raw['High'] - y5bond_raw['Low']) / y5bond_raw['Close']
y5bond_raw_data['open_close_diff_ratio'] = (y5bond_raw['Open'] - y5bond_raw['Close']) / y5bond_raw['Close']

# Add financial indicators
y5bond_stats_data = stockstats.StockDataFrame.retype(y5bond_raw_data)
y5bond_stats_data[['change','close_delta', 'close_-2_r','close_-6_r']] ### no volume

y5bond_stock = pd.DataFrame(y5bond_stats_data).drop(columns='volume').add_prefix('y5bond_')

# 10 year bonds

y10bond_raw = yf.download("^TNX", start=stock_date_start, end=stock_date_end)
y10bond_raw_data = y10bond_raw
# Add high_low_diff, open_close_diff, high_low_diff_ratio, open_close_diff_ratio
y10bond_raw_data['high_low_diff'] = (y10bond_raw['High'] - y10bond_raw['Low'])
y10bond_raw_data['open_close_diff'] = (y10bond_raw['Open'] - y10bond_raw['Close'])
y10bond_raw_data['high_low_diff_ratio'] = (y10bond_raw['High'] - y10bond_raw['Low']) / y10bond_raw['Close']
y10bond_raw_data['open_close_diff_ratio'] = (y10bond_raw['Open'] - y10bond_raw['Close']) / y10bond_raw['Close']

# Add financial indicators
y10bond_stats_data = stockstats.StockDataFrame.retype(y10bond_raw_data)
y10bond_stats_data[['change','close_delta', 'close_-2_r','close_-6_r']] ### no volume

y10bond_stock = pd.DataFrame(y10bond_stats_data).drop(columns='volume').add_prefix('y10bond_')



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [72]:
stock_without_absolute = pd.merge(stock_without_absolute, y10bond_stock[['y10bond_high_low_diff', 'y10bond_open_close_diff',
       'y10bond_high_low_diff_ratio', 'y10bond_open_close_diff_ratio',
       'y10bond_change', 'y10bond_close_delta', 'y10bond_close_-2_r',
       'y10bond_close_-6_r']], how="inner", left_index=True, right_index=True)
stock_with_absolute = pd.merge(stock_with_absolute, y10bond_stock, how="inner", left_index=True, right_index=True)

stock_without_absolute = pd.merge(stock_without_absolute, y5bond_stock[['y5bond_high_low_diff', 'y5bond_open_close_diff',
       'y5bond_high_low_diff_ratio', 'y5bond_open_close_diff_ratio',
       'y5bond_change', 'y5bond_close_delta', 'y5bond_close_-2_r',
       'y5bond_close_-6_r']], how="inner", left_index=True, right_index=True)
stock_with_absolute = pd.merge(stock_with_absolute, y5bond_stock, how="inner", left_index=True, right_index=True)

In [73]:
print("stock_without_absolute Null values? ", stock_without_absolute.isnull().values.any())
print("stock_with_absolute Null values? ", stock_with_absolute.isnull().values.any())

stock_without_absolute Null values?  False
stock_with_absolute Null values?  False


In [74]:
stock_without_absolute.describe()

,change,open_delta,close_delta,volume_delta,high_low_diff_ratio,open_close_diff_ratio,close_-2_r,close_-6_r,kdjk,kdjd,...,y10bond_close_-2_r,y10bond_close_-6_r,y5bond_high_low_diff,y5bond_open_close_diff,y5bond_high_low_diff_ratio,y5bond_open_close_diff_ratio,y5bond_change,y5bond_close_delta,y5bond_close_-2_r,y5bond_close_-6_r
count,2481.000000,2481.000000,2481.000000,2.481000e+03,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000,...,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000
mean,0.075545,0.049073,0.048956,1.434376e+05,0.017359,-0.000237,0.149953,0.440537,57.208638,57.208530,...,-0.005599,-0.003106,0.051446,0.002287,0.035191,0.001803,0.040268,-0.000228,0.076973,0.244167
std,1.439234,0.875447,0.910422,2.014634e+07,0.009092,0.011350,1.993869,3.282888,22.197408,19.166271,...,3.016018,5.207423,0.042168,0.037110,0.029581,0.026286,3.242781,0.045671,4.506002,7.791991
min,-11.399546,-6.469994,-6.099998,-2.294923e+08,0.004120,-0.053898,-12.380344,-14.475906,8.463379,11.387073,...,-16.043699,-20.364967,0.000000,-0.181000,0.000000,-0.147514,-14.468864,-0.188000,-25.458899,-28.919330
25%,-0.674506,-0.290001,-0.280006,-6.460000e+06,0.011464,-0.006757,-0.932067,-1.370401,40.438621,43.223542,...,-1.777059,-3.228290,0.030000,-0.017000,0.018944,-0.010936,-1.752579,-0.028000,-2.441270,-4.326332
50%,0.050863,0.030001,0.020000,-3.579000e+05,0.015238,-0.000386,0.186441,0.528480,59.456322,59.019476,...,-0.175977,-0.269266,0.043000,0.003000,0.028930,0.001488,0.000000,0.000000,-0.109708,-0.220266
75%,0.822480,0.370001,0.360001,6.376400e+06,0.020433,0.005704,1.267888,2.351033,76.430105,73.041141,...,1.603056,2.858476,0.063000,0.023000,0.043931,0.014925,1.706774,0.027000,2.275193,3.931844
max,10.452235,5.170006,6.599998,2.391179e+08,0.085416,0.061138,12.950242,15.401249,95.333442,92.557744,...,13.694958,25.518784,1.349000,0.234000,0.844180,0.250535,16.634976,0.230000,24.809877,41.527915


In [53]:
stock_with_absolute.describe()

,change,open_delta,close_delta,volume_delta,high_low_diff_ratio,open_close_diff_ratio,close_-2_r,close_-6_r,kdjk,kdjd,...,y5bond_close,y5bond_adj close,y5bond_high_low_diff,y5bond_open_close_diff,y5bond_high_low_diff_ratio,y5bond_open_close_diff_ratio,y5bond_change,y5bond_close_delta,y5bond_close_-2_r,y5bond_close_-6_r
count,11.000000,11.000000,11.000000,1.100000e+01,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,...,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000
mean,1.051018,2.198181,2.932728,-8.785909e+05,0.023524,-0.005790,1.716835,2.512464,65.011284,59.653905,...,0.367909,0.367909,0.029091,-0.003182,0.079142,-0.008474,1.255311,0.003909,1.701198,1.196854
std,1.743904,3.261737,4.887672,1.169045e+07,0.008188,0.011560,1.975065,4.227669,16.566217,12.880865,...,0.013678,0.013678,0.006818,0.016005,0.018750,0.043121,5.754515,0.020964,5.913426,7.587011
min,-1.620944,-3.709991,-4.729980,-2.676220e+07,0.014536,-0.020391,-1.551406,-3.814660,37.534866,42.776564,...,0.345000,0.345000,0.020000,-0.029000,0.054200,-0.078591,-7.980051,-0.032000,-9.725687,-13.126493
25%,-0.158430,0.490021,-0.434998,-1.854850e+06,0.020366,-0.011748,0.123942,-0.583363,55.585700,48.179759,...,0.363000,0.363000,0.024500,-0.015000,0.064012,-0.039018,-2.439021,-0.009000,0.000000,-3.434063
50%,1.414881,2.920013,4.089996,4.236000e+05,0.021239,-0.009019,2.481650,3.601089,70.345502,59.174988,...,0.369000,0.369000,0.027000,0.000000,0.075362,0.000000,1.373625,0.005000,1.933702,2.168026
75%,2.494982,4.914978,6.899994,4.932400e+06,0.025078,-0.002702,2.742549,5.757807,78.199478,70.676407,...,0.369000,0.369000,0.035000,0.006500,0.094851,0.017708,5.014569,0.017500,4.537690,7.564051
max,3.284523,5.889984,9.150024,1.438820e+07,0.045491,0.023333,5.463422,7.915148,83.538848,77.660894,...,0.401000,0.401000,0.039000,0.028000,0.107143,0.075881,10.164836,0.037000,10.479041,10.164836


In [75]:
stock_without_absolute.columns

Index(['change', 'open_delta', 'close_delta', 'volume_delta',
       'high_low_diff_ratio', 'open_close_diff_ratio', 'close_-2_r',
       'close_-6_r', 'kdjk', 'kdjd', 'kdjj', 'macd', 'macds', 'macdh', 'rsi_6',
       'rsi_12', 'wr_6', 'wr_12', 'cci', 'atr', 'dma', 'vr', 'boll_-1_d',
       'boll_ub_-1_d', 'boll_lb_-1_d', 'boll_k_diff', 'high_low_diff',
       'open_close_diff', 'news_title_score', 'news_des_score',
       'sp500_high_low_diff', 'sp500_open_close_diff',
       'sp500_high_low_diff_ratio', 'sp500_open_close_diff_ratio',
       'sp500_change', 'sp500_close_delta', 'sp500_volume_delta',
       'sp500_close_-2_r', 'sp500_close_-6_r', 'gold_high_low_diff',
       'gold_open_close_diff', 'gold_high_low_diff_ratio',
       'gold_open_close_diff_ratio', 'gold_change', 'gold_close_delta',
       'gold_volume_delta', 'gold_close_-2_r', 'gold_close_-6_r',
       'y10bond_high_low_diff', 'y10bond_open_close_diff',
       'y10bond_high_low_diff_ratio', 'y10bond_open_close_diff_rati

In [21]:
# store training
stock_without_absolute.to_pickle('./data/'+target+'/stock_without_absolute.pkl')
stock_with_absolute.to_pickle('./data/'+target+'/stock_with_absolute.pkl')

In [22]:
label_abs_1d = label_abs_1d.loc[stock_with_absolute.index]
label_abs_7d = label_abs_7d.loc[stock_with_absolute.index]
label_abs_30d = label_abs_30d.loc[stock_with_absolute.index]

label_value_1d = label_value_1d.loc[stock_with_absolute.index]
label_value_7d = label_value_7d.loc[stock_with_absolute.index]
label_value_30d = label_value_30d.loc[stock_with_absolute.index]

In [23]:
# store labels
label_abs_1d.to_pickle('./data/'+target+'/label_abs_1d.pkl')
label_abs_7d.to_pickle('./data/'+target+'/label_abs_7d.pkl')
label_abs_30d.to_pickle('./data/'+target+'/label_abs_30d.pkl')

label_value_1d.to_pickle('./data/'+target+'/label_value_1d.pkl')
label_value_7d.to_pickle('./data/'+target+'/label_value_7d.pkl')
label_value_30d.to_pickle('./data/'+target+'/label_value_30d.pkl')